<a href="https://colab.research.google.com/github/shofwatunniswah/SentimentAnalysisOnDanantara/blob/main/Scrap_Danantara_TERBARU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Langkah 1: Pengaturan Awal (Dilakukan di Terminal VS Code)

Sebelum menjalankan notebook, pastikan Anda sudah membuat dan mengaktifkan lingkungan virtual. Kemudian, install semua library yang dibutuhkan melalui terminal VS Code, bukan di dalam sel notebook.

In [ ]:
# Buka terminal di VS Code (Ctrl + `) dan jalankan:
.\.venv\Scripts\activate
pip install pandas newspaper3k beautifulsoup4 requests lxml_html_clean

Langkah 2: Import Library

Di sel pertama notebook Anda, mulailah dengan mengimpor semua library yang akan digunakan. Ini adalah praktik yang baik untuk menjaga kode tetap terorganisir.

In [1]:
# Import library yang dibutuhkan
import pandas as pd
import newspaper
from newspaper import Config
from urllib.parse import urlparse

print("Library berhasil diimpor.")

Library berhasil diimpor.


Langkah 3: Memuat Daftar URL dari File Lokal
Kode ini sekarang akan membaca file CSV dari folder lokal (misalnya, folder data). Anda tidak perlu lagi melakukan drive.mount.

In [2]:
try:
    file_path = 'data/Link_Berita_Gojek.csv' # Path lokal, bukan path Google Drive
    df_urls = pd.read_csv(file_path)

    # Ambil list URL dari kolom yang sesuai (misalnya, kolom 'link')
    list_of_urls = df_urls['link'].tolist()

    print(f"Berhasil memuat {len(list_of_urls)} URL dari {file_path}")
    print("Contoh URL pertama:", list_of_urls[0])
    
except FileNotFoundError:
    print(f"Error: File tidak ditemukan di path '{file_path}'. Pastikan file dan folder sudah benar.")

Berhasil memuat 179 URL dari data/Link_Berita_Gojek.csv
Contoh URL pertama: https://beritajatim.com/tarif-ojol-naik-15-persen-pakar-transportasi-ingatkan-dampak-negatif/


Langkah 4: Proses Scraping Artikel Berita
Ini adalah inti dari proses scraping. Saya menambahkan konfigurasi user_agent yang terkadang dapat membantu menghindari error 403 Forbidden saat scraping.

In [3]:
# Konfigurasi untuk newspaper3k agar terlihat seperti browser biasa
config = Config()
config.browser_user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
config.request_timeout = 30

scraped_articles = []

print("Memulai proses scraping...")
# Loop melalui setiap URL untuk mengambil datanya
for url in list_of_urls:
    try:
        article = newspaper.Article(url=url, language='id', config=config)
        article.download()
        article.parse()

        # Ekstrak nama domain sebagai sumber berita
        source = urlparse(url).netloc.replace('www.', '')

        # Kumpulkan detail artikel dalam sebuah dictionary
        article_detail = {
            "title": article.title,
            "authors": ", ".join(article.authors) if article.authors else None,
            "source": source,
            "published_date": article.publish_date,
            "summary": article.summary,
            "content": article.text,
            "url": url,
        }
        scraped_articles.append(article_detail)

    except newspaper.ArticleException as e:
        print(f"Gagal memproses URL: {url}. Error: {e}")
    except Exception as e:
        print(f"Terjadi error tak terduga untuk URL {url}: {e}")

print(f"\nProses scraping selesai. Berhasil mendapatkan data dari {len(scraped_articles)} artikel.")

# Ubah list of dictionaries menjadi DataFrame
df_scraped = pd.DataFrame(scraped_articles)
df_scraped.head()

Memulai proses scraping...
Gagal memproses URL: https://diorama.suaramerdeka.com/tech/18315140565/gojek-error-hari-ini-13-mei-2025-begini-cara-mengatasinya. Error: Article `download()` failed with 403 Client Error: Forbidden for url: https://diorama.suaramerdeka.com/tech/18315140565/gojek-error-hari-ini-13-mei-2025-begini-cara-mengatasinya on URL https://diorama.suaramerdeka.com/tech/18315140565/gojek-error-hari-ini-13-mei-2025-begini-cara-mengatasinya
Gagal memproses URL: https://diorama.suaramerdeka.com/tech/18315293390/gojek-error-hari-ini-5-juni-2025-cara-perbaiki-masalah-yang-ada. Error: Article `download()` failed with 403 Client Error: Forbidden for url: https://diorama.suaramerdeka.com/tech/18315293390/gojek-error-hari-ini-5-juni-2025-cara-perbaiki-masalah-yang-ada on URL https://diorama.suaramerdeka.com/tech/18315293390/gojek-error-hari-ini-5-juni-2025-cara-perbaiki-masalah-yang-ada
Gagal memproses URL: https://lipsus.kompas.com/pameranotomotifnasional2024/read/2024/04/07/1230

,title,authors,source,published_date,summary,content,url
0,"Tarif Ojol Naik 15 Persen, Pakar Transportasi ...",Rahajeng Kartika,beritajatim.com,2025-07-24 22:49:04+00:00,,Yogyakarta (beritajatim.com)– Pemerintah melal...,https://beritajatim.com/tarif-ojol-naik-15-per...
1,Account Suspended,None,beritanusantara.co,None,,,https://beritanusantara.co/kenaikan-tarif-ojol...
2,"Penelitian Menyebutkan, 75% Pengguna Ojol Tola...",Tim Sindonews,daerah.sindonews.com,None,,"Penelitian Menyebutkan, 75% Pengguna Ojol Tola...",https://daerah.sindonews.com/artikel/makassar/...
3,Gojek Siap Kaji Perubahan Tarif Ojek Online Me...,Harian Jogja Digital Media,ekbis.harianjogja.com,2025-07-02 00:00:00,,Gojek Siap Kaji Perubahan Tarif Ojek Online Me...,https://ekbis.harianjogja.com/read/2025/07/02/...
4,"Kenaikan Tarif Ojol Sarat Risiko, Pemerintah P...","Rio Sandy Pradana, Penulis, Editor",ekonomi.bisnis.com,2025-07-01 15:07:28+07:00,,RISED menilai penerapan kenaikan tarif ojol di...,https://ekonomi.bisnis.com/read/20250701/98/18...


Langkah 5: Pembersihan Data dan Penambahan Kolom 'Tag'
Setelah scraping, kita gabungkan kembali dengan data 'tag' dari file asli dan membersihkan data yang tanggal publikasinya kosong.

In [5]:
# Gabungkan kolom 'Label Category' dari DataFrame asli ke DataFrame hasil scraping
# Ini mengasumsikan urutan barisnya masih sama
if not df_scraped.empty:
    df_scraped['tag'] = df_urls['Label Category']

    # Hitung jumlah data sebelum dibersihkan
    rows_before = len(df_scraped)
    print(f"Jumlah baris sebelum pembersihan: {rows_before}")

    # Hapus baris yang kolom 'published_date'-nya kosong (NaT)
    df_final = df_scraped.dropna(subset=['published_date'])

    # Hitung jumlah data setelah dibersihkan
    rows_after = len(df_final)
    print(f"Jumlah baris setelah menghapus tanggal kosong: {rows_after}")
    print(f"Jumlah baris yang dihapus: {rows_before - rows_after}")
else:
    print("Tidak ada data yang berhasil di-scrape, DataFrame kosong.")
    df_final = pd.DataFrame() # Buat DataFrame kosong jika tidak ada hasil

df_final.head()

Jumlah baris sebelum pembersihan: 172
Jumlah baris setelah menghapus tanggal kosong: 103
Jumlah baris yang dihapus: 69


,title,authors,source,published_date,summary,content,url,tag
0,"Tarif Ojol Naik 15 Persen, Pakar Transportasi ...",Rahajeng Kartika,beritajatim.com,2025-07-24 22:49:04+00:00,,Yogyakarta (beritajatim.com)– Pemerintah melal...,https://beritajatim.com/tarif-ojol-naik-15-per...,Local News
3,Gojek Siap Kaji Perubahan Tarif Ojek Online Me...,Harian Jogja Digital Media,ekbis.harianjogja.com,2025-07-02 00:00:00,,Gojek Siap Kaji Perubahan Tarif Ojek Online Me...,https://ekbis.harianjogja.com/read/2025/07/02/...,Local News
4,"Kenaikan Tarif Ojol Sarat Risiko, Pemerintah P...","Rio Sandy Pradana, Penulis, Editor",ekonomi.bisnis.com,2025-07-01 15:07:28+07:00,,RISED menilai penerapan kenaikan tarif ojol di...,https://ekonomi.bisnis.com/read/20250701/98/18...,Local News
5,Gojek Pastikan Ikuti Kebijakan Kenaikan Tarif ...,"Asfian Nur Muhammad, Oleh",ekonomi.rakyat.news,2025-07-02 10:06:15+08:00,,"RAKYAT NEWS, JAKARTA – Induk perusahaan Gojek,...",https://ekonomi.rakyat.news/read/146273/gojek-...,Economic
6,"GoJek, Grab Agree to Increase Online Ojek Tariff",None,en.tempo.co,1970-01-01 07:00:00+07:00,,"TEMPO.CO, Jakarta - Transportation Ministry`s ...",https://en.tempo.co/read/917359/gojek-grab-agr...,Economic


Langkah 6: Simpan Hasil ke File CSV Lokal

In [6]:
# Simpan hasil akhir ke file CSV baru di folder 'data'
if not df_final.empty:
    output_path = 'data/Hasil_Scraping_Gojek.csv'
    df_final.to_csv(output_path, index=False)
    print(f"Data akhir berhasil disimpan di: {output_path}")
else:
    print("Tidak ada data untuk disimpan.")

Data akhir berhasil disimpan di: data/Hasil_Scraping_Gojek.csv
